In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 1
!hostname
!pwd

dv001.ib.bridges2.psc.edu
/ocean/projects/asc170022p/mtragoza/mre-pinn/notebooks


In [2]:
import sys, os, pathlib
import numpy as np
import xarray as xr
import torch
import matplotlib.pyplot as plt
import seaborn as sns

os.environ['DDEBACKEND'] = 'pytorch'
import deepxde

sys.path.append('..')
%aimport mre_pinn

torch.cuda.is_available()

Using backend: pytorch



True

In [3]:
%autoreload

cohort = mre_pinn.data.PatientCohort(xarray_dir='../data/NAFLD', patient_ids='0006')
cohort.load_xarrays()

  0%|          | 0/1 [00:00<?, ?it/s]Loading ../data/NAFLD/0006/t1_pre_in.nc
Loading ../data/NAFLD/0006/t1_pre_water.nc
Loading ../data/NAFLD/0006/t1_pre_out.nc
Loading ../data/NAFLD/0006/t1_pre_fat.nc
Loading ../data/NAFLD/0006/t2.nc
Loading ../data/NAFLD/0006/mre_raw.nc
Loading ../data/NAFLD/0006/wave.nc
Loading ../data/NAFLD/0006/mre.nc
Loading ../data/NAFLD/0006/mask.nc
100%|██████████| 1/1 [00:00<00:00, 20.86it/s]


In [4]:
anat_sequences = ['t1_pre_water', 't1_pre_out', 't1_pre_fat', 't2']
a = np.stack([
    cohort.patients['0006'].arrays[seq].values for seq in anat_sequences
], axis=-1)[None,...]
a = torch.tensor(a, device='cuda', dtype=torch.float32)
print('a', a.shape)

u = cohort.patients['0006'].arrays['wave'].values[None,...,None]
u = torch.tensor(u, device='cuda', dtype=torch.float32)
print('u', u.shape)

a torch.Size([1, 256, 256, 32, 4])
u torch.Size([1, 256, 256, 32, 1])


In [5]:
%autoreload

net = mre_pinn.pino.PINO(
    n_input=len(anat_sequences),
    n_blocks=4,
    n_hidden=16,
    n_modes=8,
    n_output=1,
    activ_fn='g'
)
net

PINO(
  (fc_input): Linear(in_features=4, out_features=16, bias=True)
  (block0): PINOBlock(
    (conv): SpectralConv3d()
    (fc): Linear(in_features=16, out_features=16, bias=True)
  )
  (block1): PINOBlock(
    (conv): SpectralConv3d()
    (fc): Linear(in_features=16, out_features=16, bias=True)
  )
  (block2): PINOBlock(
    (conv): SpectralConv3d()
    (fc): Linear(in_features=16, out_features=16, bias=True)
  )
  (block3): PINOBlock(
    (conv): SpectralConv3d()
    (fc): Linear(in_features=16, out_features=16, bias=True)
  )
  (fc_output): Linear(in_features=16, out_features=1, bias=True)
)

In [6]:
net(a).shape

  F_out[:,:n,:n,:n,:] = torch.einsum(



torch.Size([1, 256, 256, 32, 1])